## MP-3 Project

In [2]:
import pandas as pd, numpy as np, seaborn as sbn, matplotlib.pyplot as plt

from Modules import utils as utl
from Modules import data_exploration as de
from Modules import machine_learning as ml

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm

In [4]:
import pickle

### Data Loading & Cleaning

In [6]:
csv_file_path = 'Data/song_data.csv'
eurovision_df = utl.load_csv(csv_file_path, skiprows=0, encoding='windows-1252')

In [7]:
eurovision_df

,year,semi_final,semi_draw_position,final_draw_position,country,artist_name,song_name,language,style,direct_qualifier_10,...,final_jury_votes,final_place,final_total_points,semi_place,semi_televote_points,semi_jury_points,semi_total_points,favourite_10,race,host_10
0,2023,1,1,20,Norway,Alessandra,Queen of Kings,English,Pop,0,...,11.0,5.0,268.0,6.0,102.0,NaN,102.0,0,unknown,0
1,2023,1,2,NaN,Malta,The Busker,Dance (Our Own Party),English,Pop,-,...,NaN,NaN,NaN,15.0,3.0,NaN,3.0,0,unknown,0
2,2023,1,3,5,Serbia,Luke Black,Samo mi se spava,"Serbian, English",Pop,0,...,6.0,24.0,30.0,10.0,37.0,NaN,37.0,0,unknown,0
3,2023,1,4,NaN,Latvia,Sudden Lights,Aija,English,Rock,-,...,NaN,NaN,NaN,11.0,34.0,NaN,34.0,0,unknown,0
4,2023,1,5,2,Portugal,Mimicat,Ai coração,Portuguese,Pop,0,...,9.0,23.0,59.0,9.0,74.0,NaN,74.0,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,2009,-,-,3,France,Patricia Kaas,Et s'il fallait le faire,French,Ballad,1,...,NaN,8.0,218.0,NaN,NaN,NaN,NaN,0,unknown,0
561,2009,-,-,10,Russia,Anastasiya Prikhodko,Mamo,"Russian, Ukrainian",Ballad,1,...,NaN,11.0,185.0,NaN,NaN,NaN,NaN,0,unknown,1
562,2009,-,-,17,Germany,Alex Swings Oscar Sings!,Miss Kiss Kiss Bang,English,Pop,1,...,NaN,20.0,91.0,NaN,NaN,NaN,NaN,0,unknown,0
563,2009,-,-,23,United Kingdom,Jade Ewen,It's My Time,English,Ballad,1,...,NaN,5.0,328.0,NaN,NaN,NaN,NaN,0,unknown,0


In [8]:
eurovision_df.isna().sum()

year                       0
semi_final                 0
semi_draw_position         0
final_draw_position       11
country                    0
artist_name                0
song_name                  0
language                   0
style                      0
direct_qualifier_10        0
gender                     0
main_singers               0
age                        0
selection                  0
key                        0
BPM                        0
energy                     0
danceability               0
happiness                  0
loudness                  90
acousticness               0
instrumentalness           0
liveness                   0
speechiness                0
release_date               0
key_change_10              0
backing_dancers            0
backing_singers            0
backing_instruments        0
instrument_10              0
qualified_10               0
final_televote_points    237
final_jury_points        237
final_televote_votes     332
final_jury_vot

##### The dataset has numerous null values, however this is more due to the nature of the Eurovision Song Contest rather than faulty data. For instance, all the contestants of each year who performed in the semi-finals but never made it through to the final will have null values in all final related features. While some semi-final features must also be expected to be null due to some nations being automatically qualified to the final, each year.

In [10]:
finalists_df = utl.remove_missing(eurovision_df, columns='final_place')

##### Now we have two dataframes. eurovision_df with all the data and finalists_df for only the songs that made it to the final.

In [12]:
finalists_df.isna().sum()

year                       0
semi_final                 0
semi_draw_position         0
final_draw_position        0
country                    0
artist_name                0
song_name                  0
language                   0
style                      0
direct_qualifier_10        0
gender                     0
main_singers               0
age                        0
selection                  0
key                        0
BPM                        0
energy                     0
danceability               0
happiness                  0
loudness                  39
acousticness               0
instrumentalness           0
liveness                   0
speechiness                0
release_date               0
key_change_10              0
backing_dancers            0
backing_singers            0
backing_instruments        0
instrument_10              0
qualified_10               0
final_televote_points     32
final_jury_points         32
final_televote_votes     128
final_jury_vot

##### Now to remove some columns we don't need.

In [14]:
columns_to_remove = [
    'artist_name', 'song_name', 'BPM', 'energy', 'danceability', 'happiness',
    'loudness', 'acousticness', 'instrumentalness', 'liveness', 'speechiness',
    'backing_dancers', 'backing_singers', 'backing_instruments', 'instrument_10',
    'race', 'favourite_10', 'host_10', 'age', 'gender', 'main_singers', 'key',
    'key_change_10', 'selection', 'release_date', 'direct_qualifier_10', 'qualified_10',
    'language'
]

In [15]:
eurovision_clean_df = eurovision_df.drop(columns=columns_to_remove)
finalists_clean_df = finalists_df.drop(columns=columns_to_remove)

In [16]:
eurovision_clean_df.sample(5)

,year,semi_final,semi_draw_position,final_draw_position,country,style,final_televote_points,final_jury_points,final_televote_votes,final_jury_votes,final_place,final_total_points,semi_place,semi_televote_points,semi_jury_points,semi_total_points
249,2016,1,8,-,San Marino,Pop,NaN,NaN,NaN,NaN,NaN,NaN,12.0,49.0,19.0,68.0
260,2016,2,1,20,Latvia,Pop,63.0,69.0,13.0,14.0,15.0,132.0,8.0,68.0,64.0,132.0
440,2012,-,-,20,Germany,Pop,125.0,98.0,NaN,NaN,8.0,223.0,NaN,NaN,NaN,NaN
435,2012,-,-,1,United Kingdom,Ballad,36.0,11.0,NaN,NaN,25.0,47.0,NaN,NaN,NaN,NaN
127,2019,1,12,25,Australia,Opera,131.0,153.0,30.0,24.0,9.0,284.0,1.0,140.0,121.0,261.0


In [17]:
eurovision_clean_df.to_csv('Data/eurovision_cleaned.csv', index=False)

In [18]:
more_columns_to_remove = [
    'semi_final', 'semi_draw_position', 'semi_place', 'semi_televote_points',
    'semi_jury_points', 'semi_total_points'
]

In [19]:
finalists_clean_df = finalists_clean_df.drop(columns=more_columns_to_remove)

In [20]:
finalists_clean_df.sample(5)

,year,final_draw_position,country,style,final_televote_points,final_jury_points,final_televote_votes,final_jury_votes,final_place,final_total_points
297,2015,26,Albania,Pop,4.0,93.0,12.0,2.0,17.0,97.0
253,2016,24,Austria,Pop,120.0,31.0,27.0,7.0,13.0,151.0
432,2012,12,Norway,Dance,16.0,24.0,NaN,NaN,26.0,40.0
171,2018,17,Finland,Pop,23.0,23.0,4.0,6.0,25.0,46.0
539,2009,14,Malta,Ballad,18.0,87.0,NaN,NaN,22.0,105.0


In [21]:
finalists_clean_df.to_csv('Data/finalists_cleaned.csv', index=False)

### Data Exploration

In [23]:
eurovision_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   565 non-null    int64  
 1   semi_final             565 non-null    object 
 2   semi_draw_position     565 non-null    object 
 3   final_draw_position    554 non-null    object 
 4   country                565 non-null    object 
 5   style                  565 non-null    object 
 6   final_televote_points  328 non-null    float64
 7   final_jury_points      328 non-null    float64
 8   final_televote_votes   233 non-null    float64
 9   final_jury_votes       233 non-null    float64
 10  final_place            358 non-null    float64
 11  final_total_points     358 non-null    float64
 12  semi_place             483 non-null    float64
 13  semi_televote_points   243 non-null    float64
 14  semi_jury_points       212 non-null    float64
 15  semi_t

In [24]:
eurovision_clean_df.describe()

,year,final_televote_points,final_jury_points,final_televote_votes,final_jury_votes,final_place,final_total_points,semi_place,semi_televote_points,semi_jury_points,semi_total_points
count,565.000000,328.000000,328.000000,233.000000,233.000000,358.000000,358.000000,483.000000,243.000000,212.000000,483.000000
mean,2015.653097,90.539634,90.655488,15.184549,15.107296,13.391061,164.849162,9.192547,68.263374,67.575472,97.788820
std,4.312691,86.852074,74.096517,12.828622,9.868602,7.430208,143.603444,5.069863,50.617148,47.545328,73.844662
min,2009.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,2012.000000,25.750000,36.000000,4.000000,7.000000,7.000000,68.250000,5.000000,24.500000,26.750000,43.000000
50%,2016.000000,59.000000,71.000000,11.000000,13.000000,13.000000,124.500000,9.000000,54.000000,58.000000,77.000000
75%,2019.000000,127.500000,126.000000,26.000000,22.000000,20.000000,225.250000,13.500000,104.500000,98.250000,134.000000
max,2023.000000,439.000000,382.000000,42.000000,39.000000,27.000000,758.000000,19.000000,204.000000,222.000000,403.000000


In [25]:
finalists_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 358 entries, 0 to 564
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   358 non-null    int64  
 1   final_draw_position    358 non-null    object 
 2   country                358 non-null    object 
 3   style                  358 non-null    object 
 4   final_televote_points  326 non-null    float64
 5   final_jury_points      326 non-null    float64
 6   final_televote_votes   230 non-null    float64
 7   final_jury_votes       230 non-null    float64
 8   final_place            358 non-null    float64
 9   final_total_points     358 non-null    float64
dtypes: float64(6), int64(1), object(3)
memory usage: 30.8+ KB


In [26]:
finalists_clean_df.describe()

,year,final_televote_points,final_jury_points,final_televote_votes,final_jury_votes,final_place,final_total_points
count,358.000000,326.000000,326.000000,230.000000,230.000000,358.000000,358.000000
mean,2015.715084,90.503067,90.490798,15.243478,15.152174,13.391061,164.849162
std,4.312362,86.871036,74.025265,12.815514,9.831256,7.430208,143.603444
min,2009.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2012.000000,26.000000,36.000000,4.250000,7.000000,7.000000,68.250000
50%,2015.500000,59.000000,71.000000,11.000000,13.000000,13.000000,124.500000
75%,2019.000000,126.750000,125.750000,26.000000,22.000000,20.000000,225.250000
max,2023.000000,439.000000,382.000000,42.000000,39.000000,27.000000,758.000000


### Data loading, cleaning and exploration of semifinal data (removal of Big-5 countries and host country)

In [28]:
esc_sf_df = utl.remove_missing(eurovision_clean_df, columns='semi_place')

In [29]:
esc_sf_df.describe()

,year,final_televote_points,final_jury_points,final_televote_votes,final_jury_votes,final_place,final_total_points,semi_place,semi_televote_points,semi_jury_points,semi_total_points
count,483.000000,254.000000,254.000000,180.000000,180.000000,277.000000,277.000000,483.000000,243.000000,212.000000,483.000000
mean,2015.645963,95.279528,94.633858,16.038889,15.900000,12.624549,172.599278,9.192547,68.263374,67.575472,97.788820
std,4.314998,88.828056,71.306801,12.550396,9.556126,6.895954,142.041642,5.069863,50.617148,47.545328,73.844662
min,2009.000000,0.000000,7.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,2012.000000,31.000000,41.000000,6.000000,8.000000,7.000000,78.000000,5.000000,24.500000,26.750000,43.000000
50%,2016.000000,62.500000,78.000000,12.000000,15.000000,13.000000,132.000000,9.000000,54.000000,58.000000,77.000000
75%,2019.000000,133.250000,126.000000,26.250000,22.250000,18.000000,229.000000,13.500000,104.500000,98.250000,134.000000
max,2023.000000,439.000000,382.000000,42.000000,39.000000,26.000000,758.000000,19.000000,204.000000,222.000000,403.000000


In [30]:
esc_sf_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 483 entries, 0 to 559
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   483 non-null    int64  
 1   semi_final             483 non-null    object 
 2   semi_draw_position     483 non-null    object 
 3   final_draw_position    472 non-null    object 
 4   country                483 non-null    object 
 5   style                  483 non-null    object 
 6   final_televote_points  254 non-null    float64
 7   final_jury_points      254 non-null    float64
 8   final_televote_votes   180 non-null    float64
 9   final_jury_votes       180 non-null    float64
 10  final_place            277 non-null    float64
 11  final_total_points     277 non-null    float64
 12  semi_place             483 non-null    float64
 13  semi_televote_points   243 non-null    float64
 14  semi_jury_points       212 non-null    float64
 15  semi_total_

In [31]:
more_columns_to_remove_semivers = [
    'final_draw_position', 'final_televote_points', 'final_jury_points', 'final_televote_votes', 'final_jury_votes', 'final_place', 'final_total_points'
]

In [32]:
esc_sf_clean_df = esc_sf_df.drop(columns=more_columns_to_remove_semivers)

In [33]:
esc_sf_clean_df.to_csv('Data/semifinalists_cleaned.csv', index=False)

In [34]:
esc_sf_clean_df.sample(5)

,year,semi_final,semi_draw_position,country,style,semi_place,semi_televote_points,semi_jury_points,semi_total_points
261,2016,2,2,Poland,Pop,6.0,131.0,20.0,151.0
92,2021,1,16,Malta,Pop,1.0,151.0,174.0,325.0
388,2013,2,13,Norway,Dance,3.0,NaN,NaN,120.0
105,2021,2,13,Bulgaria,Ballad,3.0,101.0,149.0,250.0
466,2011,2,7,Moldova,Rock,10.0,NaN,NaN,54.0


In [35]:
esc_sf_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 483 entries, 0 to 559
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  483 non-null    int64  
 1   semi_final            483 non-null    object 
 2   semi_draw_position    483 non-null    object 
 3   country               483 non-null    object 
 4   style                 483 non-null    object 
 5   semi_place            483 non-null    float64
 6   semi_televote_points  243 non-null    float64
 7   semi_jury_points      212 non-null    float64
 8   semi_total_points     483 non-null    float64
dtypes: float64(4), int64(1), object(4)
memory usage: 37.7+ KB


In [36]:
esc_sf_clean_df.describe()

,year,semi_place,semi_televote_points,semi_jury_points,semi_total_points
count,483.000000,483.000000,243.000000,212.000000,483.000000
mean,2015.645963,9.192547,68.263374,67.575472,97.788820
std,4.314998,5.069863,50.617148,47.545328,73.844662
min,2009.000000,1.000000,0.000000,0.000000,0.000000
25%,2012.000000,5.000000,24.500000,26.750000,43.000000
50%,2016.000000,9.000000,54.000000,58.000000,77.000000
75%,2019.000000,13.500000,104.500000,98.250000,134.000000
max,2023.000000,19.000000,204.000000,222.000000,403.000000
